# Define Regions of Interest (ROIs)

In [1]:
%load_ext autoreload
%autoreload 2

import os
import pickle as pkl
from pathlib import Path

import napari
from magicgui import magicgui

from plex_pipe.core_definition.viewer_utils import (
    display_saved_rois,
    save_rois_from_viewer,
)
from plex_pipe.utils.im_utils import get_org_im_shape, get_all_resolutions, get_small_image
from plex_pipe.core_definition.roi_utils import xywh_to_corners

from plex_pipe.utils.config_loaders import load_analysis_settings

## Read in config

In [2]:
# load analysis configuration
config_path = r'../examples/example_pipeline_config.yaml'

config = load_analysis_settings(config_path)

2026-02-12 18:08:11.261 | WARNING  | plex_pipe.processors.base:_log_missing_optional_params:49 - InstansegSegmenter.Params: Parameter overlap not provided. Using default value: 80.


## Read in data

In [5]:
# read in image data

# simplify variable names
im_level = config.roi_definition.im_level
save_path = config.roi_info_file_path

# define the image path
image_path = Path(config.general.image_dir) / config.roi_definition.detection_image

# read in the requested image level or all resolution levels if not specified
if im_level == None:
    im_list = get_all_resolutions(image_path)
    im_level = 0
else:
    im_list = [get_small_image(image_path, req_level = int(im_level))]

# get the original image shape
org_im_shape = get_org_im_shape(image_path)

## Create a Napari viewer and define ROIs manually

In [ ]:
viewer = napari.Viewer()
viewer.add_image(im_list, name = 'signal')


edge_width = 2

# add a red rectangle to frame the image
frame_rect = xywh_to_corners([0,0,im_list[0].shape[1],im_list[0].shape[0]])
viewer.add_shapes(frame_rect, edge_color='white', face_color = 'transparent', shape_type='rectangle', edge_width=edge_width, name = 'frame')

# add a layer for the saved rois
display_saved_rois(viewer, IM_LEVEL = im_level, save_path = save_path, edge_width=edge_width)

#########################################################################################
# add a slider for edge width
@magicgui(
    auto_call=True,
    width_slider={"widget_type": "Slider", "min": 0, "max": 200, "label": "Edge Width"}
)
def edge_width_control(width_slider: int = edge_width):
    global edge_width
    for layer in viewer.layers:
        # Check if the layer is a Shapes layer
        if isinstance(layer, napari.layers.Shapes):
            layer.edge_width = width_slider
            layer.refresh()

    edge_width = width_slider
          
# Add the widget to the napari dock console
viewer.window.add_dock_widget(edge_width_control, area='left')

#########################################################################################
# add diplay saved button
@magicgui(auto_call=False, call_button="Display Saved ROIs", result_widget=False)
def display_saved_rois_button(viewer: napari.Viewer):
    display_saved_rois(viewer, IM_LEVEL = im_level, save_path = save_path)

# Add widget to viewer
viewer.window.add_dock_widget(display_saved_rois_button, area='left')

#########################################################################################
# add a save button
@magicgui(auto_call=False, call_button="Save ROIs")
def save_button(viewer: napari.Viewer):
    save_rois_from_viewer(viewer, org_im_shape = org_im_shape, req_level = im_level, save_path = save_path)

# Add widget to viewer
viewer.window.add_dock_widget(save_button, area='left')



# Get ROIs suggested by the SAM2 model

Execute the cells below to get suggestions from SAM model about the cores position. 

Sometimes there is no immediate feedback of how SAM2 progresses. In this situation it's expected to wait 1-2 min for the outcome.

In [6]:
import subprocess
import platform
import numpy as np

from plex_pipe.utils.config_loaders import load_workstation_config

from plex_pipe.utils.file_utils import (
    change_to_wsl_path
)
from plex_pipe.core_definition.roi_utils import (
    get_refined_rectangles,
    prepare_poly_df_for_saving,
    get_visual_rectangles,
)

from plex_pipe.core_definition.viewer_utils import (
    redo_bbox_layer,
    redo_cores_layer
)

In [7]:
# load pipeline configuration
sam_config = load_workstation_config(r'../examples/example_sam_config.yaml')
sam_config

{'model_path': '/mnt/d/sam2_test/sam2',
 'sam_env': '/home/kasia/miniforge3/envs/sam2-env/bin/python'}

## Define sampling

In [8]:
#########################################################################################
nominal_small_size = 80  #the "ideal" size of the smallest objects you expect.
min_delta_factor = 0.75 # the multiplier to find the absolute minimum (e.g., 0.5×nominal).
max_delta_factor = 1.25 # the multiplier to find the absolute maximum (e.g., 2.0×nominal)."
#########################################################################################

n_sampling_points = 2 * int((np.max(im_list[0].shape) / (nominal_small_size)))
print(f'For objects of size: {nominal_small_size}, number of points per side for SAM2 segmentation will be: {n_sampling_points}')

For objects of size: 80, number of points per side for SAM2 segmentation will be: 4


## SAM2 Segmentation

In [9]:
# read image stretching parameters from the viewer
clims = viewer.layers['signal'].contrast_limits
display_level = viewer.layers['signal']._data_level
res_level = int(display_level + im_level)# if im_level is not zero than display level will be zero

# determine the operating system
is_windows = platform.system() == "Windows"

# handle paths
im_path = str(image_path.resolve())
out_path = os.path.abspath('masks.pkl')

if is_windows:
    im_path_wsl = change_to_wsl_path(str(image_path.resolve()))
    out_path_wsl = change_to_wsl_path(os.path.join(os.getcwd(), 'masks.pkl'))

# define arguments
args = [
    "--image_path", im_path_wsl if is_windows else im_path,
    "--req_level", str(res_level),
    "--int_min", str(clims[0]),
    "--int_max", str(clims[1]),
    "--model_path", sam_config['model_path'],
    "--output_file", out_path_wsl if is_windows else out_path,
    "--points_per_side", str(n_sampling_points)
]

# define subprocess command
if is_windows:
    command = ["wsl", sam_config['sam_env'], "-m", "plex_pipe.core_definition.suggest_cores"] + args
else:
    python_exe = sam_config.get('sam_env', 'python')
    command = [python_exe, "-m", "plex_pipe.core_definition.suggest_cores"] + args

# run execution
process = subprocess.Popen(
    command,
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    text=True
)

# process subprocess messages and errors
while True:
    output = process.stdout.readline()
    if output == "" and process.poll() is not None:
        break
    if output:
        print(output, end="")

errors = process.stderr.read()
if errors:
    print(f"ERROR: {errors}")

Input image: /mnt/d/plex-pipe/examples/input/sample_1.0.4_R000_DAPI__FINAL_F.tiff
Results will be saved to: /mnt/d/plex-pipe/notebooks/masks.pkl

Preparing RGB image for segmentation...
Segmenting image. It should take around 1 min. Started at 2026-02-05 21:28:10...
Saving masks...
ERROR: /home/kasia/miniforge3/envs/sam2-env/lib/python3.11/site-packages/dask/dataframe/__init__.py:31: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed in a future version. Set the configuration option `dataframe.query-planning` to `True` or None to enable the new Dask Dataframe implementation and silence this warning.
  warnings.warn(
/home/kasia/miniforge3/envs/sam2-env/lib/python3.11/site-packages/xarray_schema/__init__.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.


## Filter and display masks

In [10]:
# masks processing & display

# load the masks
masks = pkl.load(open(out_path, 'rb'))
print(f'Initial number of masks: {len(masks)}')

# refine masks
rect_list = get_refined_rectangles(masks, im = im_list[display_level],
                                   min_area = (min_delta_factor * nominal_small_size/(2**display_level))**2,
                                   max_area = (max_delta_factor * nominal_small_size/(2**display_level))**2,
                                   min_iou = 0.8,
                                   min_stability = 0.9,
                                   min_int = np.mean(im_list[display_level]) # only masks of objects of intensity above this threshold are retained
                                   )
print('Number of refined masks:', len(rect_list))

# prepare df
df = prepare_poly_df_for_saving(rect_list, poly_types = ['rectangle']*len(rect_list), org_im_shape = org_im_shape, req_level = res_level)
rect_list = get_visual_rectangles(df, im_level)
poly_list = [(x/(2**im_level)).astype('int') for x in df.polygon_vertices.to_list()]

# add to napari
redo_bbox_layer(viewer,rect_list,df['roi_name'].tolist(), edge_width = edge_width)
redo_cores_layer(viewer,rect_list,shape_type = df.poly_type.to_list(), edge_width = edge_width)

# make cores layer active
viewer.layers.selection.clear()
viewer.layers.selection.add(viewer.layers['ROIs'])

Initial number of masks: 3
Number of refined masks: 2


In [10]:
# cleanup intermediate SAM2 result
os.remove(out_path)

In [12]:
import pandas as pd
pd.read_pickle('../examples/output/sample_analysis/rois.pkl')

,roi_name,row_start,row_stop,column_start,column_stop,poly_type,polygon_vertices
0,ROI_000,256.0,5056.0,256.0,5120.0,rectangle,"[[256.0, 5120.0], [5056.0, 5120.0], [5056.0, 2..."
1,ROI_001,128.0,4992.0,6912.0,11776.0,rectangle,"[[128.0, 11776.0], [4992.0, 11776.0], [4992.0,..."
